# Uniquely mapping through subsystem Density Functional Theory (DFT)

## Table of contents:

1. [Introduction](#introduction)
2. [Martyna-Tuckerman](#paragraph 1)
3. [Uniquely mapping](#paragraph 2)
4. [Impurity problem](#paragraph 3)
5. [Ionizationpotential](#paragraph 4)
6. [Calculations](#paragraph 5)
7. [Literature](#paragraph 7)


## Introduction  <a name="introduction"></a>

This is a jupyter notebook to summarize the basics of an embedding approach developed in the Pavanello group to map periodic systems onto a collection finite systems. This method may open the field to a variety of interesting properties of systems within a crystal surrounding.

For this we are using a method introduced by Martyna and Tuckerman.

> http://aip.scitation.org/doi/abs/10.1063/1.477923

## Martyna-Tuckerman Method  <a name="paragraph 1"></a>

Martyna-Tuckerman introduced a method based on a reciprocal space formalism for treating longrange interactions in cluster systems. Through this method interactions from periodic images with the unit cell are cut off.
>http://aip.scitation.org/doi/abs/10.1063/1.3662863

It is important to mention that the coulomb operator is still periodic and therefore standart FFT techniques can be used.

This is achieved by introducing a screening potential which screens out interactions of the unit cell with neighbouring cells. 

$\hat{\phi}^{\mathrm(screen)} = \bar{\phi}^{\mathrm{(long)}} - \tilde{\phi}^{\mathrm{(long)}}$


It is clear that only the longrange part of the coulomb interaction need to be considered in this case. In the screening potential the difference between the fourier series and the fourier transform of the longrange part of the coulomb interaction cuts of the interaction of neighbouring cells. It is assumed that in case of the short range interaction both quantities are the same when the unit cell is chosen large enough.

## Uniquely mapping <a name="paragraph 2"></a>


We are introducing a new formalism which allows the mapping of periodic potentials onto a collection of finite systems.
This is achieved by calculating an appropriate embedding potential $v_\mathrm{emb}$ of the crystal surrounding for one isolated fragment. For this purpose it is useful to calculate this $v_\mathrm{emb}$ using a subsystem-DFT approach with periodic boundary conditions.

A naiv implementation of this idea would be to calculate $v_\mathrm{emb}$ by subtracting the fragment for which we want to calculate $v_\mathrm{emb}$

$v_{\mathrm{emb,I}} = v_\mathrm{J,H}[\mathrm{\rho_{env}}] + v_\mathrm{J,ext}(\mathrm{r}_{\mathrm{env}})\quad \mathrm{with} \quad \mathrm{J} \: \forall \: \mathrm{N_{s}} \: \notin \: \mathrm{I}$.

Through excluding the fragment I from $v_\mathrm{H}$ and $v_\mathrm{ext}$ all periodic replicas of this fragment would be excluded as well and the effective potential of the surrounding isn't represented by $v_\mathrm{emb,I}$ correctly.

Through the Martyna-Tuckerman method we get the possibility to subtract an isolated fragment from the periodic system.

$v_\mathrm{emb,I} = v_\mathrm{tot}[\rho_\mathrm{tot}] - v_\mathrm{MT,I}[\rho_{\mathrm{I}}]$

This leads to an effective potential of the surrounding. The verification for this becomes clear, if we look at impact of the $v_\mathrm{emb,I}$ onto a finite calculation of only the subsystem I.

$v_\mathrm{finite,tot,I} = v_\mathrm{finite,I} + v_\mathrm{emb,I}$

$\Leftrightarrow v_\mathrm{finite,tot,I} = v_\mathrm{finite,I}[\rho_{\mathrm{I}}] + v_\mathrm{tot}[\rho_\mathrm{tot}] - v_\mathrm{MT,I}[\rho_{\mathrm{I}}]$

$\Leftrightarrow v_\mathrm{finite,tot,I} = v_\mathrm{tot}[\rho_\mathrm{tot}]$

## The impurity problem <a name="paragraph 3"></a>


For the calculation of ionizationpotentials through mapping of properties two appropriate $v_\mathrm{eff,I}$ are needed. In the case for a not ionized fragment one can use the map introduced above.

In the case of the ionized fragment the potential $v_\mathrm{tot}[\rho_\mathrm{tot}]$ must be a potential which only includes one 'impurity' or in other words only one ionized fragment. As we have seen in the unique map this is again a not trivial task in periodic boundary conditions. For this reason we introduce a 'half coupled' method to calculate $v_\mathrm{tot}[\rho_\mathrm{tot}]$ in the impurity case.

For this we introduce a special workflow and screening potentials to achieve this goal. The screening potential $\hat{\phi}^{\mathrm(screen)}_\mathrm{I}$ is calculated as following:

$\hat{\phi}^{\mathrm(screen)}_\mathrm{I} = \phi_\mathrm{H, I} - \phi_\mathrm{H, MT, I} $

In this case we are only looking at the Hartree-Potential $\phi_\mathrm{H, I}$ of fragment I because the local potential is for both the ionized case and the not ionized case the same. This screening function can be interpreted as the interaction of the surrounding onto one fragment I.

Our idea is to calculate $\hat{\phi}^{\mathrm(screen)}_\mathrm{I}$ for the not ionized system at the end of the calculation. In the next step this is imported in a calculation with the ionized fragment.

During each SCF iterations $\hat{\phi}^{\mathrm(screen)}_\mathrm{I}$ is also calculated (on the large cell). Furthermore the Difference between both is calculated:

$\Delta \hat{\phi}^{\mathrm(screen)}_\mathrm{I} = \hat{\phi}^{\mathrm(screen, 0)}_\mathrm{I} - \hat{\phi}^{\mathrm(screen, ionized)}_\mathrm{I}$.

$\Delta \hat{\phi}^{\mathrm(screen)}_\mathrm{I}$ can be seen as an function which is replacing the surrounding which is ionized with a surrounding of a not ionized fragment. Of course this is a simplification because we are only taking into account coulomb interactions and only the effect of the surrounding onto the fragment and not vice versa. Furthermore The interaction of the other fragments (in the unit cell and the surrounding) are still interacting with the ionized fragment.

The procedure is leading to the following 'impurity'-potential.

$v_\mathrm{imp} = v_\mathrm{eff, I}^{\mathrm{ion}} + \Delta \hat{\phi}^{\mathrm(screen)}_\mathrm{I}$

In our case we think, although there are a few assumption taken into account, this mehtod should deliver statisfactoring results.

## Ionizationpotential <a name="paragraph 4"></a>

Through the introduction of the unique map and the impurity potential the we are geting the possibility to calculate the vertical ionizationpotentials from total energy differences, although ionizationpotentials are hard to define in calculations with periodic boundary conditions (pbc). This is caused by the fact that the ionized fragment is in every periodicly repeating unit cell of the system and the ionizationpotential is only properly defined in the limit of an infinite large supercell.

Furthermore energy differences of systems with different total charge are difficult to define.

> http://aip.scitation.org/doi/abs/10.1063/1.477923

The reason for this is that the ewald summation of a total charged periodic system is diverging.

> http://www.sciencedirect.com/science/article/pii/0021999187900763 (maybe take something different)

To avoid divergency in the ewald summation QE is adding an opposite charged background to the system. Through  this backroung energy eigenvalues are shifted and energy differences lead to wrong results. A workaround this problem is to define a proper reference system for both systems.

> http://aip.scitation.org/doi/abs/10.1063/1.464650

In our case we are distinguishing two different reference systems.

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!**Carefull probably wrong look at Embedding IP Johannes for more info** !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
* crystal reference:

The derivation of the crystal reference system is performed from the effective potentials of subsystem I. 

$1)\quad v_\mathrm{eff, I}^{0} (g = 0) = 0$

$2)\quad v_\mathrm{emb, I}^{\mathrm{ion}} (g = 0) = v_\mathrm{imp, I}^{\mathrm{ion}} (g = 0) = v_\mathrm{eff, I}^{\mathrm{ion}}(g = 0) + \Delta \hat{\phi}^{\mathrm(screen)}_\mathrm{I} (g = 0) $

with $v_\mathrm{eff, I}^{\mathrm{ion}}(g = 0) = 0$


$\Rightarrow  v_\mathrm{imp, I}^{\mathrm{ion}} (g = 0) = \phi_\mathrm{H, MT, I}^\mathrm{ion} - \phi_\mathrm{H, MT, I}^0 $

* vacuum reference:

In the other case we are referencing both systems in the final calulation to the vacuum level of the isolated (Martyna-Tuckerman) systems through adding:

$1)\Rightarrow \quad v_\mathrm{eff, I}^{0} (g = 0) + \phi_\mathrm{H, MT, I}^0 (g = 0)+ \phi_\mathrm{loc, MT, I}^0(g = 0) = \phi_\mathrm{MT, I}^0(g = 0)$

$2)\Rightarrow \quad v_\mathrm{imp, I}^{\mathrm{ion}} (g = 0) + \phi_\mathrm{H, MT, I}^0 (g = 0)+ \phi_\mathrm{loc, MT, I}^{\mathrm{ion}} (g = 0) + \phi_\mathrm{H, MT, I}^0 (g = 0) = \phi_\mathrm{MT, I}^\mathrm{ion}(g = 0)$

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

### Workflow:


<img src="Workflow.png" width="220" height="320'">

## Calculations <a name="paragraph 6"></a>
Three test systems were chosen to proof the applicability of our method. In both cases we are looking at ionic crystals.

### System 1: CsI

#### Calculations with eQE


* Computational details:
    * Cubic cell with a  = 4.567 Å for (two subsystem case)
    > https://journals.aps.org/prb/abstract/10.1103/PhysRevB.33.8706
    * Ecutwfc = 75 Ry
    * Ecutrho = 750 Ry
    * fde_kin_funct = 'LC94'
    * K -Points: 1 1 1 0 0 0
    * use_gaussians = .true.
    * only no cell split for the 128 for the subsystem of interest


| Subsytems    | vtot         | ionized     | impurity     | MT           | MT ionized   | map          |map ionized| map vaccum| map vacuum ionized|
| ------------ |:------------:|:------------:|:------------:|:------------:|:------------:|:------------:|:---------:|:---------:|:---------:|
| 2            |  x           | x            | x            | x            | x            | x            | x    |x|x|     |
| 16           |  x           | x            | x            | x             | x            | x            | x    |x|x|    |
| 54           |  x           | x            | x            | x            | x            | x            | x    |x|x    |
| 128 cellsplit(1,2,3) = 0.6         |  x           | x            | x            | x            | x            | x            | x         |x|x|
| 128 cellsplit(1,2,3) = 1.0         |  x           | x            | x            | x            | x            | x            | x         |x|x|

### System 2: NaCl 

#### Calculations with eQE


* Computational details:
    * Cell parameters from: 
    > https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.84.1942
    * Ecutwfc = 75 Ry
    * Ecutrho = 750 Ry
    * fde_kin_funct = 'LC94'
    * K -Points: 1 1 1 0 0 0


| Subsytems    | vtot         | ionized     | impurity     | MT           | MT ionized   | map          |map ionized| map vaccum| map vacuum ionized|
| ------------ |:------------:|:------------:|:------------:|:------------:|:------------:|:------------:|:---------:|:---------:|:---------:|
| 2            |  x           | x            | x            | x            | x            | x            | x    |x|x|     |
| 16           |  x           | x            | x            | x            | x            | x            | x    |x|x|    |
| 54           |  x           | x            | x            | x            | x            | x            | x    |x|x    |
| 128          |  x           | x            | x            | x            | x            | x            | x         |x|x|



### System 3: Cs2UO2Cl4 / Model C

#### Calculations with eQE


* Computational details:
    * Monoclinic cell: 
    
        a = (11.93, 0.00, -1.01)
        
        b = (0.00, 7.70, 0.00)
        
        c = (0.00, 0.00, 5.73)
        
    > http://scripts.iucr.org/cgi-bin/paper?s0108270191006777
    * Ecutwfc = 95 Ry
    * Ecutrho = 950 Ry
    * fde_kin_funct = 'LC94'
    * K -Points: 1 1 1 0 0 0
    * use_gaussians = .true.

| Subsytems    | vtot         | ionized      | impurity     | MT           | MT ionized   | map          |map ionized| map vacuum| map vacuum charged|
| ------------ |:------------:|:------------:|:------------:|:------------:|:------------:|:------------:|:---------:|:---------:|:---------:|
| 14            |  x           | x            | x            | x            | x            | x            | x |x|x|      
| 84           |  x           | x            | x            | x            | x            | x            | x  |x|x|       
|Model C| x           | x            | x            | x            | x            | x            | x         |x|x|


## Literature: <a name="paragraph 7"></a>

1) Papers from Peter A. Schulz:

--> Method for the calculation of ionizatiopotential of NaCl

--> Theory of defect levels and the "Band Gap Problem" in Silicon

> https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.84.1942

> https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.96.246401

> https://journals.aps.org/prb/abstract/10.1103/PhysRevB.60.1551